In [ ]:
import pandas as pd

url = "https://gist.githubusercontent.com/guilhermesilveira/1b7d5475863c15f484ac495bd70975cf/raw/16aff7a0aee67e7c100a2a48b676a2d2d142f646/projects.csv"

df = pd.read_csv(url)
df.head()

In [ ]:
# Queremos trabalhar com os nomes das tabelas em português, portanto renomearemos os títulos por meio do método rename(), 
# passando paras as colunas as definições que escreveremos em uma variável a_renomear.

a_renomear = {
    'expected_hours' : 'horas_esperadas',
    'price' : 'preco',
    'unfinished' : 'nao_finalizado'
}
df = df.rename(columns = a_renomear)
df.head()

In [ ]:
# Contudo, a coluna nao_finalizado pode gerar confusões, afinal 0 seriam os projetos finalizados e 1 os não finalizados. 
# Podemos organizar as nossas features com mais clareza de forma a evitar erros no desenvolvimento do projeto.
# Criaremos uma nova coluna chamada finalizados, e trocaremos os valores 0 e 1, invertendo a função deles nesta coluna.

troca = {
    0 : 1,
    1 : 0
}
df['finalizado'] = df.nao_finalizado.map(troca)
df.head()

In [ ]:
# O que queremos agora é visualizar as informações de outra forma. 
# No projeto anterior, havia três features e um y. 
# Neste caso, temos duas features (horas_esperadas e preco) e a classificação (finalizado).

# Com essas duas features, podemos desenhar em um eixo as horas_esperadas e em outro o preco. 
# Para isso, usaremos uma famosa biblioteca do Python chamada Seaborn. 
# Vamos importá-la como sns (que é a importação padrão), e plotaremos os pontos distribuídos por meio do scatterplot().

# Para este método, precisamos passar o parâmetro referente às coluna x (horas_esperadas) e y (preco). 
# Por fim, devemos passar os dados, de onde vem nosso dataframe do Pandas.

import seaborn as sns

sns.scatterplot(x="horas_esperadas", y="preco", data=df)


In [ ]:
# Existem várias maneiras de tentarmos visualizar as duas classes (0 ou 1 para finalizado) no gráfico. Uma delas é por meio de cores. 
# Utilizaremos a mesma estrutura do método scatterplot, mas dessa vez passando o parâmetro hue, que será baseado na coluna finalizado, pintando os pontos de acordo com a classe atribuída à essa coluna:

sns.scatterplot(x="horas_esperadas", y="preco", hue="finalizado", data=df)


In [ ]:
sns.relplot(x="horas_esperadas", y="preco", col="finalizado", data=df)

In [ ]:
sns.relplot(x="horas_esperadas", y="preco", hue="finalizado", col="finalizado", data=df)


In [ ]:
# Nosso próximo passo é realizar a modelagem. 
# Primeiro, coletaremos nossos dados no eixo x, que são as colunas horas_esperadas e preco. 
# Em seguida, o eixo y, referente à coluna finalizado.

x = df[['horas_esperadas', 'preco']]
y = df['finalizado']


In [ ]:
# Agora criaremos um modelo, separaremos os dados de treino e teste, treinaremos efetivamente o modelo e por fim verificaremos a taxa de acerto.

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

train_x, test_x, train_y, test_y = train_test_split(x, y, random_state = 42, test_size = 0.25,stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(train_x), len(test_x)))

modelo = LinearSVC()
modelo.fit(train_x, train_y)
previsoes = modelo.predict(test_x)

acuracia = accuracy_score(test_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

In [ ]:
# Para obtermos uma resposta satisfatória, precisamos de um recurso comparativo. 
# Portanto, inventaremos um algoritmo bem simples que terá como previsão que todos os projetos são finalizados, isto é, os 540 elementos de teste serão 1.
# Com o Numpy, a biblioteca que gera matrizes, faremos com que 540 elementos tenham o valor 1. 
# Feito isso, estipularemos que essas são as previsões, as chamaremos de previsoes_sam, e testaremos sua acurácia.

import numpy as np
previsoes_sam = np.ones(540)
acuracia = accuracy_score(test_y, previsoes_sam) * 100
print("A acurácia foi %.2f%%" % acuracia)

In [ ]:
# Agora iremos analisar cada uma das classificações que realizamos. 
# Para isso, copiaremos o mesmo código de scatterplot() que utilizamos anteriormente, mas, ao invés de df, usaremos test_x. 
# Também precisaremos alterar o campo que serve de parâmetro para hue - no caso, test_y.

sns.scatterplot(x="horas_esperadas", y="preco", hue=test_y, data=test_x) 


In [ ]:
x_min = test_x.horas_esperadas.min()
x_max = test_x.horas_esperadas.max()
y_min = test_x.preco.min()
y_max = test_x.preco.max()
print(x_min, x_max,y_min,y_max)

In [ ]:
# O x está indo de 1 a 100, enquanto y varia de 101 a 27738. 
# Agora definiremos quantos pixels haverá em cada dimensão; 
# se multiplicamos 100 x 100, isto é, 100 pixels para cada dimensão (altura e largura), # teremos um total de 10.000 pontos, o que é o suficiente.
# Entre x_min e x_max, cada espaço terá o tamanho máximo do gráfico dividido pelo total de pixels ((x_max - x_min)/pixels.
# Adicionaremos o método arange() da biblioteca do Numpy.

pixels = 100
np.arange(x_min, x_max, (x_max - x_min)/pixels)

In [ ]:
# A partir de 1, foram realizadas somas de 0.99 até chegar ao número 99.01, dividindo proporcionalmente o espaço do eixo_x no gráfico. 
# Faremos o mesmo procedimento para o eixo_y, aproveitando para nomear cada uma dessas operações:

pixels = 100
eixo_x = np.arange(x_min, x_max, (x_max - x_min)/ pixels)
eixo_y = np.arange(y_min, y_max, (y_max - y_min)/ pixels)


In [ ]:
# Agora a ideia é criarmos um grid entre os dois eixos, multiplicando as opções de acordo com as diferentes possibilidade de custo de projeto e tempo de execução. 
# Temos na biblioteca do Numpy o np.meshgrid(), que consegue mesclar um grid . 
# Esse método recebe como parâmetros o eixo_x e eixo_y, e devolve o conteúdo é xx e yy. Vamos analisar xx:

xx, yy = np.meshgrid(eixo_x, eixo_y)
xx

In [ ]:
# Com o código estruturado desta maneira, o que temos é a repetição do eixo x 100 vezes, e o mesmo ocorre com eixo y.
#  Ou seja, ainda não mesclamos realmente esses dados. Para fazermos isso, usaremos o xx.ravel():

xx, yy = np.meshgrid(eixo_x, eixo_y)
xx.ravel()

In [ ]:
# Faremos o mesmo procedimento para yy, e então concatenaremos xx e yy por meio de np.c_. Essa operação nos devolverá os pontos.

xx, yy = np.meshgrid(eixo_x, eixo_y)
pontos = np.c_[xx.ravel(), yy.ravel()]
pontos

In [ ]:
# Agora, nosso trabalho é, a partir do modelo, fazer as previsões (predict()) para todos esses pontos. 
# Como resultado, teremos uma série de números, que são as classificações. Chamaremos essas classificações de Z.

Z = modelo.predict(pontos)

In [ ]:
# Precisamos nos atentar para o formato (shape) do array, que é 10000, mas se analisarmos xx, verificaremos que suas dimensões são 100, 100

Z.shape

In [ ]:
xx.shape

In [ ]:
# Redimensionar (reshape) o array de 10000 de acordo com xx.
Z = Z.reshape(xx.shape)
Z

In [ ]:
# Plotar esses dados. 
# Para isso, usaremos a biblioteca Matplotlib.
# Novamente, queremos que os pontos sejam espalhados (scatter()), mas não usaremos o scatterplot() do Seaborn pois, nesse caso, queremos um controle mais refinado dessa plotagem.

import matplotlib.pyplot as plt

plt.scatter(test_x.horas_esperadas, test_x.preco, c=test_y)


In [ ]:
# Reduzir o tamanho dos pontos e melhorar a visualização dos dados, usar o parâmetro s=1 (de size):

import matplotlib.pyplot as plt

plt.scatter(test_x.horas_esperadas, test_x.preco, c=test_y, s=1)


In [ ]:
# Já temos o plot básico, a próxima tarefa é desenhar a classificação que estipulamos: se o valor for 0, pintaremos de uma cor, se for 1, de outra.
# Além disso, entre esses números, adicionar uma cor diferente para visualizar a curva que separa as duas classificações do algoritmo.
# Para tanto, usar plt.contourf() (de countor, ou "contorno"), que receberá xx, yy e Z. 
# Para que a tonalidade não seja muito forte, escrever alpha=0.3, aumentando a transparência.

plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(test_x.horas_esperadas, test_x.preco, c=test_y, s=1)


## Neste codigo aprendemos a:

 - Trocar valores;
 - Usar a biblioteca seaborn;
 - Gerar um gráfico com dados do CSV;
 - Definir cores no gráfico;
 - Gerar gráficos distintos;
 - Criação de um algoritmo base;
 - Capturar os valores mínimos e máximos.
 - Utilizar a função arange do numpy